In [1]:
from typing import List, Optional
import json
import os
import sys

if "linux" in sys.platform:
    os.environ["LC_AL"] = "C.UTF-8"
    os.environ["LANG"] = "C.UTF-8"
else:
    os.environ["LC_AL"] = "en_US.utf-8"
    os.environ["LANG"] = "en_US.utf-8"

import numpy as np
import tweepy
from aitextgen import aitextgen

In [2]:
def new_auth():
    consumer_key = "..."
    consumer_secret = "..."
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    redirect_url = auth.get_authorization_url()
    print(redirect_url)

In [3]:
def get_access_token():
    oauth_token = "..."
    oauth_verifier = "..."
    auth.request_token = { 'oauth_token' : oauth_token,
                           'oauth_token_secret' : oauth_verifier }
    print(auth.get_access_token(oauth_verifier))

In [4]:
def authenticate(consumer_key=None, consumer_secret=None, access_key=None, access_secret=None):
    consumer_key = consumer_key or os.environ["CONSUMER_KEY"]
    consumer_secret = consumer_secret or os.environ["CONSUMER_SECRET"]
    access_key = access_key or os.environ["ACCESS_KEY"]
    access_secret = access_secret or os.environ["ACCESS_SECRET"]

    # Authenticate with the Twitter API
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    return api

In [5]:
import logging
logging.basicConfig(
        format="%(asctime)s — %(levelname)s — %(name)s — %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.DEBUG
    )
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [6]:
ai = aitextgen(model="pytorch_model.bin",
               config="config.json",
               vocab_file="aitextgen-vocab.json",
               merges_file="aitextgen-merges.json",
               to_gpu=False)

INFO:aitextgen:Loading GPT-2 model from provided pytorch_model.bin.
INFO:aitextgen:Using a custom tokenizer.


In [20]:
ai.generate(n=50, prompt="@quemerogoku")

@quemerogoku

@quemerogoku

@quemerogoku

@quemerogokuooo ¿que mierda sale por verla?"

@quemerogokuoo

@quemerogoku

@quemerogoku?

@quemerogoku

@quemerogokuoo

@quemerogoku"

@quemerogoku

@quemerogoku"

@quemerogoku "mastientan la cara de Rubre la de la universidad de Asunk

@quemerogoku https://t.co/CrxWFLpGpZ

@quemerogoku https://t.co/Mh5oZ4sSyV

@quemerogoku "si le doy un toldo"

@quemerogoku que me da miedo?

@quemerogoku"

@quemerogokuooo https://t.co/kSpwGo3oEy

@quemerogoku

@quemerogoku

@quemerogoku (23) &lt;3) http://t.co/895wZr2yhD

@quemerogoku

@quemerogoku?" "el marcos nos hacen la noche o no es la peor idea que la verdad que no le gusta a los mosquitos"

@quemerogoku

@quemerogokuooo

@quemerogoku? #seaperolteral

@quemerogoku"

@quemerogoku" #unpicastenach

@quemerogoku

@quemerogoku oficial en el subte?

@quemerogoku!

@quemerogoku que me hace malos http://t.co/Yj2KqTV055

@quemerogoku

@quemerogokuoo

@quemerogoku (-si le

@quemerogoku?

@quemerogokuooo https://t

In [15]:
import re
def new_tweet_text(temperature=0.7):
    # generate a tweet
    generated_text = ai.generate_one(temperature=temperature)
    while len(generated_text) >= 280:
        generated_text = ai.generate_one(temperature=temperature)
    
    if 'https://t.co' in generated_text:
        generated_text = re.sub('https\:\/\/t\.co.*', '', generated_text)
    
    print(generated_text)
    return generated_text

In [66]:
api = authenticate()

In [62]:
api.update_status(new_tweet_text())

la verdad que no voy a empezar a hacer el único nombre de un tipo que no sea a que no le vas a hacer todo a la mierda



Status(_api=<tweepy.api.API object at 0x7f8b63b87fd0>, _json={'created_at': 'Sun Sep 27 03:24:20 +0000 2020', 'id': 1310057661996699648, 'id_str': '1310057661996699648', 'text': 'la verdad que no voy a empezar a hacer el único nombre de un tipo que no sea a que no le vas a hacer todo a la mierda', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="https://twitter.com" rel="nofollow">neuraltwitbot</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1310015580615909376, 'id_str': '1310015580615909376', 'name': 'Dr. Octobot', 'screen_name': 'acaestatoldomal', 'location': '', 'description': 'Soy un bot creado por @NLaski y entrenado con los tuits de @acaestatodomal', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 5, 'friends_count': 10, 'listed_coun

In [71]:
from random import randint
from time import sleep


class MyStreamListener(tweepy.StreamListener):
    def on_status(self, status):
        if status.author.id == 190461974:
            seconds = randint(60, 300)
            print(f"new tweet. will wait {seconds} seconds and then tweet")
            sleep(seconds)
            api.update_status(new_tweet_text())

myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth=api.auth, listener=myStreamListener)
myStream.filter(follow=["190461974"])
#myStream.filter(track=["python"])

KeyboardInterrupt: 